In [315]:
import pandas as pd

In [316]:
df = pd.read_csv('BTC_Trial_NB.csv')

In [ ]:
df.index = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df.head()

In [318]:
df['Return'] = df['ClosePrice '].pct_change()
df = df.dropna()

In [319]:
import pandas as pd
import numpy as np
from arch import arch_model
from sklearn.model_selection import train_test_split
from statsmodels.stats.stattools import jarque_bera
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
df['Return'].head()

In [ ]:
# Define the features and target variable
y = df['Return']

# Split the data into train and test sets
y_train, y_test = train_test_split(y, test_size=0.2, shuffle=False)

# Define and fit the GARCH-X model with scaled features
model = arch_model(y_train, vol='GARCH', p=2, q=3, o=2, mean='zero')
garch = model.fit(disp="off")

# Print summaries
print("GARCH-X Model Summary:")
print(garch.summary())

In [ ]:
history = [x for x in y_train]

# make first prediction
predictions = list()
model = arch_model(history, vol='GARCH', p=2, q=3, o=2, mean='zero')
model_fit = model.fit(disp="off")
conditional_volatility = model_fit.conditional_volatility

forecast = model_fit.forecast()
forecast = np.sqrt(forecast.variance)
yhat = forecast.T
predictions.append(yhat.values)
history.append(y_test[0])

for i in range(1, len(y_test),1):
    # predict
    model = arch_model(history, vol='GARCH', p=2, q=3, o=2, mean='zero')
    model_fit = model.fit(disp="off")
    forecast = model_fit.forecast(horizon=1)
    forecast = np.sqrt(forecast.variance)
    yhat = forecast.T
    # invert transformed prediction
    predictions.append(yhat.values)
    # observation
    obs = y_test[i]
    history.append(obs)

flattened_predictions = [item for sublist in predictions for item in (sublist if isinstance(sublist, np.ndarray) else [sublist])]

print(flattened_predictions)

In [ ]:
len(flattened_predictions), len(y_test)

In [324]:
flattened_predictions = flattened_predictions[1:]
y_test = y_test[:-1]

In [334]:
prediction_df = pd.DataFrame(flattened_predictions, columns=['Predictions'])
prediction_df = prediction_df - 0.021
prediction_df.index = y_test.index


In [ ]:
conditional_volatility.shape, y.shape

In [ ]:
len(y_test)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(16,8))
# plt.plot(y.index[-600:], y.tail(600), color='green', label = 'Train Stock Price')

p=400

# plt.plot(df.index[-600:], df['Return'].tail(600), color = 'grey', label = 'Realized Returns')
plt.plot(df.index[-p:], df['VOL1'].tail(p), color = 'blue', label = 'Actual VOL1')
plt.plot(y_test.index[-p:], prediction_df[-p:], color = 'red', label = 'GARCH')
plt.title('Volatility Prediction (GARCH)')
plt.xlabel('Time')
plt.ylabel('Volatility')
plt.legend()
plt.grid(True)
plt.show()

In [337]:
prediction_df.to_csv('GARCH.csv')

In [ ]:
# RMSE
from sklearn.metrics import mean_squared_error
import numpy as np
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# MAPE
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#NMSE
def nmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    variance = np.var(y_true)
    return mse / variance

#DA
def DA(y_true, y_pred):
    # Convert the arrays to numpy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate the direction of change
    true_direction = np.sign(np.diff(y_true))
    pred_direction = np.sign(np.diff(y_pred))
    
    # Compare directions
    correct_direction = np.sum(true_direction == pred_direction)
    total_direction = len(true_direction)
    
    # Calculate directional accuracy
    da = correct_direction / total_direction * 100
    
    return da

y_true = np.array(conditional_volatility[-len(prediction_df):])
y_predi = np.array(prediction_df).flatten() 

print("RMSE: ", rmse(y_true, y_predi))
print("MAPE: ", mape(y_true, y_predi))
print("NMSE: ", nmse(y_true, y_predi))
print("DA: ", DA(y_true, y_predi))

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 6), dpi=300)

# Plot ACF
plot_acf(df['Return'], lags=100, ax=ax[0])
ax[0].set_title('ACF (Autocorrelation Function)')
ax[0].set_ylim(-0.1, 0.1)

# Plot PACF
plot_pacf(df['Return'], lags=100, ax=ax[1])
ax[1].set_title('PACF (Partial Autocorrelation Function)')
ax[1].set_ylim(-0.1, 0.1)

plt.tight_layout()
plt.show()